In [ ]:
# # SpikeInterface v0.98
# # 0. Preparation
import spikeinterface.full as si
from pathlib import Path
import warnings
warnings.simplefilter("ignore")
from probeinterface import read_prb

In [ ]:
print(f"SpikeInterface version: {si.__version__}")

In [ ]:
# # 0.1. Setting parameters  # #
base_folder = Path('/Users/zhangjinming/Documents/Open Ephys/2_2023-11-25_13-31-27')
oe_folder = base_folder / 'Record Node 119'
probe_folder = Path('/Users/zhangjinming/Documents/OpenEphys')
Kedou16 = read_prb(probe_folder / '4X4_Kedoubc1to16.prb')
freq_min = 300
freq_max = 3000
isi_viol_thresh = 0.2
amp_cutoff_thresh = 0.5
job_kwargs = dict(n_jobs=-1, chunk_duration="1s", progress_bar=True)

In [ ]:
# # 1. Reading recording and sorting
full_raw_rec = si.read_openephys(oe_folder)
raw_rec = full_raw_rec.set_probegroup(Kedou16)
probe_rec = raw_rec.get_probe()

In [ ]:
# # 2. Preprocessing
recording_f = si.bandpass_filter(raw_rec, freq_min=freq_min, freq_max=freq_max)
recording_cmr = si.common_reference(recording_f, reference='global', operator='median')

In [ ]:
# # 3. Saving and loading SpikeInterface objects
if (base_folder / "preprocessed").is_dir():
    recording_saved = si.load_extractor(base_folder / "preprocessed")
else:
     recording_saved = recording_cmr.save(folder=base_folder / "preprocessed", **job_kwargs)

In [ ]:
# # 4. Spike sorting
# We will sort the bandpass cached filtered recording the `recording_saved` object.
get_default_params = si.get_default_sorter_params('spykingcircus2')
si.get_default_sorter_params('spykingcircus2')
# run spike sorting on entire recording
sorting_SC2 = si.run_sorter('spykingcircus2', recording_saved,
                            output_folder=base_folder / 'results_SC2',
                            verbose=True,
                            **get_default_params)
sorting_SC2 = sorting_SC2.remove_empty_units()
print(f'SC found {len(sorting_SC2.get_unit_ids())} non-empty units')
sorting_saved_SC2 = sorting_SC2.save(folder=base_folder / "sorting_SC2")

In [ ]:
# # 5. Extracting waveforms
# To extract the waveforms, we can run:
recording_saved = si.load_extractor(base_folder / "preprocessed")
sorting = sorting_SC2
we = si.extract_waveforms(recording_saved, sorting, folder=base_folder / "waveforms_dense",
                          sparse=False,
                          overwrite=True, **job_kwargs)

In [ ]:
# # 7. Quality metrics and curation
# Let's see what metrics are available:
#print(si.get_quality_metric_list())
#print(si.get_quality_pca_metric_list())
metric_names = si.get_quality_metric_list()
qm = si.compute_quality_metrics(we, metric_names=metric_names, verbose=True, **job_kwargs)
# ## Automatic curation based on quality metrics
our_query = f"amplitude_cutoff < {amp_cutoff_thresh} & isi_violations_ratio < {isi_viol_thresh}"
keep_units = qm.query(our_query)
keep_unit_ids = keep_units.index.values

In [ ]:
sorting_auto_SC2= sorting.select_units(keep_unit_ids)
print(f"Number of units before curation: {len(sorting.get_unit_ids())}")
print(f"Number of units after curation: {len(sorting_auto_SC2.get_unit_ids())}")
we_curated = we.select_units(keep_unit_ids, new_folder=base_folder / "waveforms_curated")

In [ ]:
# # 10. Exporters
# ## Export to Phy for manual curation
# To perform manual curation we can export the data to [Phy](https://github.com/cortex-lab/phy). 
si.export_to_phy(we_curated, output_folder=base_folder / 'phy_SC2',
                   compute_amplitudes=False, compute_pc_features=False, copy_binary=False,
                   **job_kwargs)
# After curating the results we can reload it using the `PhySortingExtractor`
# and exclude the units that we labeled as `noise`:
#sorting_phy_curated = se.PhySortingExtractor(base_folder / 'phy_KS25/', exclude_cluster_groups=['noise'])

In [ ]:
# ## Export a report to a folder
si.export_report(we, output_folder=base_folder / 'report_SC')